In [ ]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import IPython.display as ipd
import os
import shutil
from tqdm import tqdm
from typing import Optional, Union
from __future__ import annotations
from pprint import pprint


# Parameters
MAX_ITER = 1 * 10**5
EPOCH = 100
B = int(MAX_ITER) // EPOCH
lr = 1
p = 21

np.set_printoptions(precision=None)
np.random.seed(42)

class Tensor(object):
    def __init__(
        self, 
        x: Union[np.ndarray, cp.ndarray], 
        require_grad: bool = False,
        device: Union[cp.cuda.device.Device, None] = None,
        dtype: Union[np.dtype, cp.dtype] = cp.float32
    ):
        self.value = x
        self.grad = np.zeros_like(x) if require_grad else None
        self.require_grad = require_grad
        self.device = 'cpu' if device is None else device
    
    def __str__(self) -> str:
        return x.__str__()

class Layer(object):
    def __init__(self, name: str):
        self.name = name
        
    def backprop(self):
        raise NotImplementedError()

class Perceptron:
    def __init__(self, name, w1=None, w2=None, theta=None):
        self.w1 = w1 if w1 is not None else 1. * np.random.randn()
        self.w2 = w2 if w2 is not None else 1. * np.random.randn()
        self.theta = theta if theta is not None else 1. * np.random.randn()
        self.name = name

    def sigmoid(self, x):
        return np.divide(1, 1+np.exp(-x))

    def __call__(self, x1, x2):
        return self.sigmoid(x1*self.w1 + x2*self.w2 - self.theta)

    def backprop(self, x1, x2, g=None, dLdz=None, lr: float=.1):
        y = self(x1, x2)
        if dLdz is None and g is not None:
            dLdz = -2 * (g-y)
        dLdy = dLdz * y*(1-y)
        dLdw1 = dLdy * x1
        dLdw2 = dLdy * x2
        dLdt = - dLdy

        dLdx1 = dLdy * self.w1
        dLdx2 = dLdy * self.w2

        self.w1 -= lr * dLdw1
        self.w2 -= lr * dLdw2
        self.theta -= lr * dLdt

        return dLdx1, dLdx2

    def __str__(self) -> str:
        return self.name

    def explain(self):
        return {'w': [self.w1, self.w2], 'theta': self.theta}
    
    def plot(self, ax):
        X = np.random.uniform(-.05, 1.05, 10000)
        Y = np.random.uniform(-.05, 1.05, 10000)
        Z = self(X, Y)

        scatter = ax.scatter(X, Y, c=Z, s=.1)
        plt.colorbar(scatter, ax=ax)
        ax.set_title(str(self))

        return ax

class XOR(Perceptron):
    def __init__(self):
        self.NAND = Perceptron('P1', theta=-.7)
        self.OR = Perceptron('P2', theta=.2)
        self.AND = Perceptron('P3', theta=.7)
        self.name = 'XOR'
    
    def __call__(self, x1, x2, backprop=False):
        x1, x2 = self.NAND(x1, x2), self.OR(x1, x2)
        if backprop:
            return self.AND(x1, x2), x1, x2
        return self.AND(x1, x2)
    
    def backprop(self, x1, x2, g: np.ndarray, lr: float = 0.1):
        y, z1, z2 = self(x1, x2, backprop=True)
        dLdz1, dLdz2 = self.AND.backprop(z1, z2, g=g, lr=lr)

        self.NAND.backprop(x1, x2, dLdz=dLdz1, lr=lr)
        self.OR.backprop(x1, x2, dLdz=dLdz2, lr=lr)
    
    def explain(self):
        res = {}
        for p in [self.NAND, self.OR, self.AND]:
            res.update({str(p): p.explain()})
        pprint(res)
    
    def plot_all(self):
        fig, axes = plt.subplots(2, 2, figsize=(10, 8))

        axes[0, 0] = self.NAND.plot(axes[0, 0])
        axes[0, 1] = self.OR.plot(axes[0, 1])
        axes[1, 0] = self.AND.plot(axes[1, 0])
        axes[1, 1] = self.plot(axes[1, 1])

        plt.show()

x = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
])
g = np.array([
    [0],
    [1],
    [1],
    [0]
])

X = np.linspace(-.05, 1.05, 100)

xor = XOR()

for i in range(MAX_ITER):
    for j in range(4):
        xor.backprop(x[j,0], x[j,1], g[j][0], lr=lr)

print('for input: ')
pprint(x)
print('result: ')
pprint(xor(x[:, 0],x[:, 1]).reshape(4, 1))
print('MSE')
pprint((xor(x[:, 0],x[:, 1]).reshape(4, 1)-g)**2)

xor.plot_all()



In [3]:
import cupy as cp

A = cp.array([[0, 1, 2]])
type(A.device)

cupy.cuda.device.Device